# Rogue Trader: path differences

## Intro

### Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import re
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 300)
import warnings
warnings.filterwarnings('ignore')
import openpyxl
import xlsxwriter

### Functions

In [ ]:
def calculate_differences(df_old, df_new, item_name_col='Item_Name', column_group=None):
    """The function calculates differences between two dataframes.
    Parameters: 
    df_old - the previous table version
    df_new - the latest table version
    item_name_col - columns with item_names or ability_names or talent_names
    column_group - specify columns for the parameter "on" in merge function.
    
    """
    _diff = df_old.merge(df_new, indicator='Version',on = column_group, how = 'left',).loc[lambda x:x['Version']!='both']
    unique_names = _diff[item_name_col].unique().tolist() # list of unique names with differences
    index_values = _diff.index.values        
    try:
        _changes = (pd.concat([_diff.loc[index_values], df_new.loc[index_values]])
                            .reset_index()
                            .sort_values(by='index')                           
                        )
        # sort rows by name, previous version first
        _changes = _changes.rename(columns = {'index':'Item_Index'})
        _changes['Version'] = _changes['Version'].astype('str')
        _changes['Version'] = _changes['Version'].replace({'left_only':0,'nan':1})
        _changes = _changes.sort_values(by = [item_name_col, 'Item_Index','Version']).reset_index(drop = True)
        _changes['Version'] = _changes['Version'].replace({0:'Previous', 1:'Latest'})
    except KeyError as err:    # if keys are not found in the new table:  
                    print(f'Unexpected {err=}, {type(err)=}')
                    keys = re.findall('\d+',str(err))
                    keys = [int(i) for i in keys] # list of keys in key error message
                    index_values = [i for i in index_values if i not in keys]    # list of indexes not caused error                                    
                    # create   a common part of the table of differences
                    ## select rows of df_new with indexes in index_values and item_names in _diff['Item_Name].unique()
                    _subset = df_new[np.logical_and(df_new.index.isin(index_values), df_new[item_name_col].isin(unique_names))]
                    _changes = pd.concat([_diff.loc[index_values], _subset])                        
                    # find names of items caused error in the _diff table
                    names = _diff.loc[keys][item_name_col].unique().tolist()           
                    # select rows with those items from the df_new by name (instead of by indexes)
                    _temp = df_new[df_new[item_name_col].isin(names)]
                    _temp =pd.concat([_diff.loc[keys], _temp]) # create a problem part of the table of differences
                    #  create a one table
                    _changes = (pd.concat([_changes, _temp])
                            .reset_index()                            
                            )
                    # sort rows by name, previous version first
                    _changes = _changes.rename(columns = {'index':'Item_Index'})
                    _changes['Version'] = _changes['Version'].astype('str')
                    _changes['Version'] = _changes['Version'].replace({'left_only':0,'nan':1})
                    _changes = _changes.sort_values(by = [item_name_col, 'Version','Item_Index']).reset_index(drop = True)
                    _changes['Version'] = _changes['Version'].replace({0:'Previous', 1:'Latest'})                           
                    
    ## Do  some column arranging
    _changes = _changes.drop('Item_Index', axis = 1)
    if column_group is not None:
        column_group.insert(1,'Version')
        #column_group.insert(2,'Item_Index')
        #columns = _changes[column_group].columns.to_list()
        #columns = columns[1:2] +columns[0:1] + columns[-1:] +columns[2:-1]
        _changes = _changes[column_group]
        return _changes
    else:
        columns = _changes.columns.to_list()
        columns = columns[0:1] + columns[-1:] +  columns[1:-1]
        _changes = _changes[columns]
        return _changes

### Folders

In [ ]:
#folders
folder = r'e:\Steam\RogueTrader\SaveFolder\ReadyTables\Differences'


## Work part

### Weapons

In [ ]:
subset = ['Название', 'Описание', 'Фракция_Продавец', 'Локация', 'Семейство',
       'Тяжёлое', 'Двуручное_или_Одноручное', 'Категория', 'Редкость',
       'Тип_Урона', 'Пробитие_Уклонения', 'Скорострельность',
       'Мин_Базовый_Урон', 'Макс_Базовый_Урон', 'Боезапас', 'Макс_Дальность',
       'Ближнее_или_Дистанц', 'Пробитие_Брони', 'Отдача', 'Фактор_Урона',
       'Название_Способности_1', 'Описание_Способности_1',
       'Стоимость_ОД_Способности_1', 'Тип_Действия_Способности_1',
       'Название_Способности_2', 'Описание_Способности_2',
       'Стоимость_ОД_Способности_2', 'Тип_Действия_Способности_2',
       'Название_Способности_3', 'Описание_Способности_3',
       'Стоимость_ОД_Способности_3', 'Тип_Действия_Способности_3',
       'Название_Способности_4', 'Описание_Способности_4',
       'Стоимость_ОД_Способности_4', 'Тип_Действия_Способности_4',
       'Название_Способности_5', 'Описание_Способности_5',
       'Стоимость_ОД_Способности_5', 'Тип_Действия_Способности_5', 'Доп_Описание']

In [ ]:
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Оружие').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Оружие').fillna('')
changes_weapons = calculate_differences(df_old = df_old,
                                            df_new = df_new,
                                            item_name_col = 'Название', 
                                            column_group=subset)
display(changes_weapons.head())

#find differences sometines not working
diff_weapons = pd.concat([weapons_new,weapons_old]).drop_duplicates(keep = False)
display(diff_weapons.head())

### Armors

In [ ]:
subset = ['Название', 'Описание', 'Фракция_Продавец', 'Локация', 'Категория',
       'Редкость', 'Тип', 'Поглощение_Урона', 'Отражение_Урона',
       'Категория_Брони']
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Броня').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Броня').fillna('')
changes_armors = calculate_differences(df_old = df_old[subset],
                                        df_new = df_new[subset],
                                        item_name_col = 'Название',
                                       )
display(changes_armors)

### Shoulders

In [ ]:
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Накидки').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Накидки').fillna('')
df_new.head(1)

In [ ]:
subset = ['Название',	'Описание',	'Фракция_Продавец',	'Локация',	'Категория',	'Редкость']
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Накидки').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Накидки').fillna('')
changes_shoulders = calculate_differences(df_old = df_old,
                                            df_new = df_new,
                                            item_name_col = 'Название',
                                            column_group=subset)
display(changes_shoulders.head())

### Feet

In [ ]:
subset = ['Название',	'Описание',	'Фракция_Продавец',	'Локация',	'Категория',	'Редкость']
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Сапоги').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Сапоги').fillna('')
changes_feet = calculate_differences(df_old = df_old,
                                            df_new = df_new,
                                            item_name_col = 'Название',
                                            column_group=subset)
display(changes_feet.head())

### Gloves

In [ ]:
subset = ['Название',	'Описание',	'Фракция_Продавец',	'Локация',	'Категория',	'Редкость']
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Перчатки').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Перчатки').fillna('')
changes_gloves = calculate_differences(df_old = df_old,
                                            df_new = df_new,
                                            item_name_col = 'Название',
                                            column_group=subset)
display(changes_gloves.head())

### headgear

In [ ]:
subset = ['Название',	'Описание',	'Фракция_Продавец',	'Локация',	'Категория',	'Редкость']
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Головные уборы').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Головные уборы').fillna('')
changes_headgear = calculate_differences(df_old = df_old,
                                            df_new = df_new,
                                            item_name_col = 'Название',
                                            column_group=subset)
display(changes_headgear.head())

### Necklaces

In [ ]:
subset = ['Название',	'Описание',	'Фракция_Продавец',	'Локация',	'Категория',	'Редкость']
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Ожерелья').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Ожерелья').fillna('')
changes_neck = calculate_differences(df_old = df_old,
                                            df_new = df_new,
                                            item_name_col = 'Название',
                                            column_group=subset)
display(changes_neck.head())

### Rings

In [ ]:
subset = ['Название',	'Описание',	'Фракция_Продавец',	'Локация',	'Категория',	'Редкость']
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Кольца').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Кольца').fillna('')
changes_rings = calculate_differences(df_old = df_old,
                                            df_new = df_new,
                                            item_name_col = 'Название',
                                            column_group=subset)
display(changes_rings.head())

### Class abilities

In [ ]:
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Классовые умения').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Классовые умения').fillna('')
df_new['Описание_Способности'] = df_new['Описание_Способности'].str.strip()
df_old['Описание_Способности'] = df_old['Описание_Способности'].str.strip()
changes_class_abilities = calculate_differences(df_old = df_old,
                                                    df_new = df_new,
                                                    item_name_col = 'Название_Способности' )
display(changes_class_abilities.head())

In [ ]:
changes_class_abilities.loc[0,'Описание_Способности']==changes_class_abilities.loc[1,'Описание_Способности']

### Homeworld talents

In [ ]:
df_new = pd.read_excel(folder + r'\rogue_trader_items_ru_new.xlsx', sheet_name='Таланты происхождения').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_ru_old.xlsx', sheet_name='Таланты происхождения').fillna('')
changes_hw_talents = calculate_differences(df_old = df_old,
                                                    df_new = df_new,
                                                    item_name_col = 'Талант' )
display(changes_hw_talents.head())

### Enemy ships

df_new = pd.read_excel(folder + r'\rogue_trader_items_eng_new.xlsx', sheet_name='EnemyShips').fillna('')
df_old = pd.read_excel(folder + r'\rogue_trader_items_eng_old.xlsx', sheet_name='EnemyShips').fillna('')
changes_enemy_ships = calculate_differences(df_old = df_old,
                                                    df_new = df_new,
                                                    item_name_col = 'Ship_Name' )
display(changes_enemy_ships)

## Saving

save_folder = r'e:\Steam\RogueTrader\SaveFolder'
changes_weapons.to_excel(save_folder + r'\changes_weapons_test.xlsx', index= None)

In [ ]:
# Save folder of ready tables
save_folder = r'e:\Steam\RogueTrader\SaveFolder\ReadyTables'
# excel writer
writer = pd.ExcelWriter(save_folder + r'\rogue_trader_patch_diff_ru.xlsx')
# excel workbook
workbook = writer.book

Style format options:

In [ ]:
#headers style
header_format = workbook.add_format({
        'bold':True,
        'text_wrap':True,
        'valign':'top',
        'align':'center',
        'fg_color':'#d5c4a5'
    })
# cells style
cell_names = workbook.add_format({'text_wrap':True,
                                       'align':'center',
                                       'valign':'vcenter',
                                       'bold':True})

cell_format = workbook.add_format({'text_wrap':True,
                                       'align':'center',
                                       'valign':'vcenter'})
cell_format2 = workbook.add_format({'align':'center',
                                       'valign':'vcenter'})
# Apply conditional formatting
cell_red = workbook.add_format({'bg_color':'#9C0006',
                                'align':'center',
                                 'valign':'vcenter' })
cell_light_red = workbook.add_format({'bg_color':'#e06666',
                                'align':'center',
                                 'valign':'vcenter' })
cell_yellow = workbook.add_format({'bg_color':'#eece6c',
                                'align':'center',
                                 'valign':'vcenter' })
cell_gray = workbook.add_format({'bg_color':'#999999',
                                'align':'center',
                                 'valign':'vcenter' })
#
cell_green = workbook.add_format({'bg_color':'#b6d7a8',
                                'align':'center',
                                 'valign':'vcenter' })
cell_blue = workbook.add_format({'bg_color':'#6fa8dc',
                                'align':'center',
                                 'valign':'vcenter' })

##### Weapons table

In [ ]:
# WEAPONS
changes_weapons.to_excel(writer, sheet_name='Оружие_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Оружие_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_weapons.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:F',80/x, cell_format)
worksheet.set_column('G:G',100/x, cell_format2)
worksheet.set_column('H:H',130/x, cell_format2)
worksheet.set_column('I:I',130/x, cell_format2)
worksheet.set_column('J:U',100/x, cell_format2)
    ## ability 1 group
worksheet.set_column('V:V',85/x, cell_format)
worksheet.set_column('W:W',200/x, cell_format)
worksheet.set_column('X:X',80/x, cell_format2)
worksheet.set_column('Y:Y',92/x, cell_format2)
    ## ability 2 group
worksheet.set_column('Z:Z',85/x, cell_format)
worksheet.set_column('AA:AA',200/x, cell_format)
worksheet.set_column('AB:AB',80/x, cell_format2)
worksheet.set_column('AC:AC',92/x, cell_format2)
    ## ability 3 group
worksheet.set_column('AD:AD',85/x, cell_format)
worksheet.set_column('AE:AE',200/x, cell_format)
worksheet.set_column('AF:AF',80/x, cell_format2)
worksheet.set_column('AG:AG',92/x, cell_format2)
    ## ability 4 group
worksheet.set_column('AH:AH',85/x, cell_format)
worksheet.set_column('AI:AI',200/x, cell_format)
worksheet.set_column('AJ:AJ',80/x, cell_format2)
worksheet.set_column('AK:AK',92/x, cell_format2)
    ## ability 5 group
worksheet.set_column('AL:AL',85/x, cell_format)
worksheet.set_column('AM:AM',200/x, cell_format)
worksheet.set_column('AN:AN',80/x, cell_format2)
worksheet.set_column('AO:AO',92/x, cell_format2)
# suppot columns
worksheet.set_column('AP:AP',200/x, cell_format)
    

### Armors

In [ ]:
changes_armors.to_excel(writer, sheet_name='Броня_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Броня_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_armors.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)

## Armor stats columns
worksheet.set_column('F:K',100/x, cell_format2)

### Shoulders

In [ ]:
changes_shoulders.to_excel(writer, sheet_name='Накидки_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Накидки_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_shoulders.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

### Feet

In [ ]:
changes_feet.to_excel(writer, sheet_name='Сапоги_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Сапоги_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_feet.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

### Gloves

In [ ]:
changes_gloves.to_excel(writer, sheet_name='Перчатки_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Перчатки_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_gloves.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

### Headgear

In [ ]:
changes_headgear.to_excel(writer, sheet_name='ГоловныеУборы_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['ГоловныеУборы_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_headgear.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

### Necklaces

In [ ]:
changes_neck.to_excel(writer, sheet_name='Ожерелья_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Ожерелья_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_neck.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

### Rings

In [ ]:
changes_rings.to_excel(writer, sheet_name='Кольца_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Кольца_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_rings.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',400/x, cell_format)
worksheet.set_column('D:D',137/x, cell_format)
worksheet.set_column('E:E',170/x, cell_format)
worksheet.set_column('F:G',100/x, cell_format2)

### Class abilities

In [ ]:
changes_class_abilities.to_excel(writer, sheet_name='КлассовыеУмения_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['КлассовыеУмения_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_class_abilities.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',120/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',120/x, cell_format)
worksheet.set_column('D:D',120/x, cell_format)
worksheet.set_column('E:E',400/x, cell_format)
worksheet.set_column('F:I',120/x, cell_format)

### Homeworld abilities

In [ ]:
changes_hw_talents.head(1)

In [ ]:
changes_hw_talents.to_excel(writer, sheet_name='ТалПроисх_Изменения_Патча', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['ТалПроисх_Изменения_Патча']

#Format header
for col_num, value in enumerate(changes_hw_talents.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',120/x, cell_names)
worksheet.set_column('B:B',80/x, cell_format)
worksheet.set_column('C:C',120/x, cell_format)
worksheet.set_column('D:D',400/x, cell_format)


In [ ]:
writer.save()